In [1]:
from dotenv import load_dotenv
import helper

# Loading data from the enviroment variable
load_dotenv()

True

In [2]:
from anthropic import Anthropic


# Grabs API key from env

client = Anthropic()
model_name = "claude-sonnet-4-6"

In [ ]:
messages = []

helper.add_user_message(
    messages=messages,
    content="Generate a short event bridge rule  as json"
    )

# Contains unecessary fluff, makes copying just the answer hard
answer = helper.chat(model_name=model_name,client=client,messages=messages)

answer

'Here\'s a simple EventBridge rule in JSON format:\n\n```json\n{\n  "Name": "my-event-rule",\n  "Description": "Trigger Lambda on S3 object creation",\n  "EventPattern": {\n    "source": ["aws.s3"],\n    "detail-type": ["Object Created"],\n    "detail": {\n      "bucket": {\n        "name": ["my-bucket"]\n      }\n    }\n  },\n  "State": "ENABLED",\n  "Targets": [\n    {\n      "Id": "MyLambdaTarget",\n      "Arn": "arn:aws:lambda:us-east-1:123456789012:function:my-function"\n    }\n  ]\n}\n```\n\n### Breakdown:\n| Field | Description |\n|-------|-------------|\n| `Name` | Name of the rule |\n| `EventPattern` | Filters which events trigger the rule |\n| `source` | AWS service that emits the event |\n| `detail-type` | Type of event |\n| `State` | `ENABLED` or `DISABLED` |\n| `Targets` | Where to send the matched event (e.g., Lambda, SQS, SNS) |\n\nThis example triggers a **Lambda function** whenever an object is created in a specific **S3 bucket**.'

In [ ]:
import json

messages = []

event_bridge_schema = {
    "type": "object",
    "properties": {
        "source": {"type": "array", "items": {"type": "string"}},
        "detail-type": {"type": "array", "items": {"type": "string"}},
        "detail": {
            "type": "object",
            "properties": {},
            "additionalProperties": False 
        }
    },
    "required": ["source", "detail-type", "detail"],
    "additionalProperties": False
}

helper.add_user_message(
    messages=messages,
    content="Generate a short event bridge rule  as json"
    )


response = client.messages.create(
    model=model_name,
    max_tokens=1024,
    messages=messages,
    output_config={
        "format": {
            "type": "json_schema",
            "schema": event_bridge_schema
        }
    }
)
print(response.content[0].text)
print("----")
parsed = json.loads(response.content[0].text)
print(parsed)

{"source":["aws.ec2"],"detail-type":["EC2 Instance State-change Notification"],"detail":{}}
----
{'source': ['aws.ec2'], 'detail-type': ['EC2 Instance State-change Notification'], 'detail': {}}
